In [ ]:
aaPairs = {"A":"Ala", "R":"Arg", "N":"Asn", "D":"Asp",
           "C":"Cys", "E":"Glu", "Q":"Gln", "G":"Gly",
           "H":"His", "I":"Ile", "L":"Leu", "K":"Lys",
           "M":"Met", "F":"Phe", "P":"Pro", "S":"Ser",
           "T":"Thr", "W":"Trp", "Y":"Tyr", "V":"Val"}

aaTable = dict(list(zip(*list(zip(*aaPairs.items()))[::-1])))

In [ ]:
import pandas as pd

# Import HuRI-Union Reference Interactome

In [ ]:
huri = pd.read_csv("data/HuRI/HI-union.tsv",
                   delimiter="\t",
                   header=None)
huri.columns = ["A","B"]

# Import Processed Dataset of Variant Effect on PPI measured by Y2H

In [ ]:
y2h = pd.read_csv("data/y2hEdgotyping/y2HMerged.csv",index_col=0)

In [ ]:
y2h

In [ ]:
gene_ids = list(set(huri.A).union(set(huri.B)))

# SKIP - Get GO Terms

In [ ]:
# with open("data/HuRI/HI-union.ensg.csv","w") as f:
#     f.write("\n".join(gene_ids))

# import requests

# len(gene_ids)

# def batchiter(seq, size):
#     return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# from tqdm import tqdm

# responses = []
# for batch in tqdm(batchiter(gene_ids,100),total=len(gene_ids)//100):
#     r = requests.post("https://mygene.info/v3/gene?fields=go",data={"ids":",".join(batch)})
#     if r.status_code == 200:
#         terms = r.json()
#         responses.append(terms)
#     else:
#         print(r.status_code)

BP = Biological Process

CC = Cellular Component

MF = Molecular Function

In [ ]:
def extract(terms):
    if terms is None:
        ids = set()
    elif type(terms) is dict:
        ids = set([terms["qualifier"]+":"+terms["id"]])
    else:
        ids = set((t["qualifier"]+":"+t["id"] for t in terms))
    return ids

In [ ]:
terms = {"BP":set(), "CC": set(), "MF":set()}
gene_results = {}
nf = 0
for response in responses:
    for g in response:
        if "notfound" in g and g["notfound"]: nf += 1
        if "go" not in g: continue
        bpids = extract(g["go"]["BP"] if "BP" in g["go"] else set())
        ccids = extract(g["go"]["CC"] if "CC" in g["go"] else set())
        mfids = extract(g["go"]["MF"] if "MF" in g["go"] else set())
        g_res = {"bp":bpids,"cc":ccids,"mf":mfids}
        gene_results[g["query"]] = g_res
        terms["BP"].update(bpids)
        terms["CC"].update(ccids)
        terms["MF"].update(mfids)

In [ ]:
nf

In [ ]:
len(gene_results)

In [ ]:
len(terms["BP"])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
nbp = len(terms["BP"])
plt.hist([len(g["bp"]) for g in gene_results.values()])

# Issues with Using Gene Ontology
[Gene Ontology](http://geneontology.org/docs/go-annotations/)
* Gene products are annotated to the most granular term in the ontology that is supported by the available evidence.
* By the transitivity principle, an annotation to a GO term implies annotation to all its parents


In [ ]:
huri

In [ ]:
gene_id_map = dict(zip(list(gene_ids),range(len(gene_ids))))

In [ ]:
huri = huri.assign(ID_A=huri.A.apply(lambda ensg: gene_id_map[ensg]),
                  ID_B=huri.B.apply(lambda ensg: gene_id_map[ensg]))

In [ ]:
huri

In [ ]:
with open("/data/dzeiberg/ppi/HuRI/HI-union.edgelist","w") as f:
    for a,b in zip(huri.ID_A,huri.ID_B):
        f.write(f"{a} {b}\n")
        
with open("/data/dzeiberg/ppi/HuRI/HI-union.weightedEdgeList","w") as f:
    for a,b in zip(huri.ID_A,huri.ID_B):
        f.write(f"{a} {b} 1\n")

In [ ]:
import networkx as nx

In [ ]:
G = nx.read_weighted_edgelist("/data/dzeiberg/ppi/HuRI/HI-union.weightedEdgeList",nodetype=int)

In [ ]:
import nxmetis

In [ ]:
(cut,parts) = nxmetis.partition(G, 2)

In [ ]:
cut/len(G.edges)

In [ ]:
G_train,G_val = G.subgraph(parts[0]), G.subgraph(parts[1])

In [ ]:
len(G_train.edges),len(G_val.edges)

In [ ]:
np.mean([G_train.degree(n) for n in G_train.nodes()])

In [ ]:
np.mean([G_val.degree(n) for n in G_val.nodes()])

In [ ]:
nx.write_edgelist(G_train,"/data/dzeiberg/ppi/HuRI/HI-union.train.edgelist")

In [ ]:
nx.write_edgelist(G_val,"/data/dzeiberg/ppi/HuRI/HI-union.val.edgelist")

# Read in Node2Vec

In [ ]:
import pandas as pd

In [ ]:
n2vTrain = pd.read_csv("/data/dzeiberg/ppi/HuRI/HI-union.train.emb",
                       delimiter=" ",
                       skiprows=[0],header=None,index_col=0)

n2vVal = pd.read_csv("/data/dzeiberg/ppi/HuRI/HI-union.val.emb",
                       delimiter=" ",
                       skiprows=[0],header=None,index_col=0)

In [ ]:
n2vTrain

# Read in Mutpred2 Results

In [ ]:
mp = pd.read_csv("data/y2hEdgotyping/mutpred2Results/variants.faa.out")
mp = mp.assign(ID=mp.ID.str.replace("db_orf_",""),
              aa_change_mt=mp.Substitution.apply(lambda s: aaPairs[s[0]]+s[1:-1]+aaPairs[s[-1]]))

In [ ]:
mp

In [ ]:
mp.columns

In [ ]:
mp.loc[0,"Molecular mechanisms with Pr >= 0.01 and P < 0.99"].split(";")

# Get the file and row number for each variant's mutpred2 features

In [ ]:
u_idx = np.stack((np.ones(len(mp)),
                  np.zeros(len(mp))),axis=1)
last = mp.ID.values[0]
for i,v in enumerate(mp.ID.values[1:],start=1):
    if v == last:
        u_idx[i,0] = u_idx[i-1,0]
        u_idx[i,1] = u_idx[i-1,1] + 1
    else:
        last = v
        u_idx[i,0] = u_idx[i-1,0] + 1

In [ ]:
mp = mp.assign(featFileNum=u_idx[:,0],fileRowNum=u_idx[:,1])

In [ ]:
mp.ID = mp.ID.astype(int)

# Merge dataframes

In [ ]:
df = pd.merge(y2h,mp,left_on=["db_orf_id","aa_change_mt"],
        right_on=["ID","aa_change_mt"],validate="m:1")

In [ ]:
df

In [ ]:
#df.to_csv("data/y2hEdgotyping/y2hWithMutPred2Info.csv")

# Define Score change at level 3

In [ ]:
df = df.assign(delta3=df.LWH25_f_wt - df.LWH25_f_mt)

# Load Mutpred2 Features for each row

In [ ]:
df.db_n2v_idx.isna().any()

In [ ]:
gene_id_map

In [ ]:
df = df.assign(db_n2v_idx=df.db_ensembl_gene_id_mt.apply(lambda ensg: gene_id_map[ensg]),
              ad_n2v_idx=df.ad_ensembl_gene_id_mt.apply(lambda ensg: gene_id_map[ensg]))

In [ ]:
n2vAll = pd.concat((n2vTrain,n2vVal))

In [ ]:
def getfold(row):
    if row.db_n2v_idx in n2vTrain.index and row.ad_n2v_idx in n2vTrain.index:
        return 0
    elif row.db_n2v_idx in n2vVal.index and row.ad_n2v_idx in n2vVal.index:
        return 1
    return 2

In [ ]:
from scipy.io import loadmat

X = np.zeros((len(df),1345))
foldNum = np.zeros(len(df))
for idx_i in df.featFileNum.astype(int).unique():
    idxmask = df.featFileNum == idx_i
    rownums = df.fileRowNum[idxmask].astype(int)
    f = loadmat(f"data/y2hEdgotyping/mutpred2Results/variants.faa.out.feats_{idx_i}")["feats"]
    X[idxmask,:] = f[rownums]
    foldNum[idxmask] = df.loc[idxmask].apply(lambda row: getfold(row),axis=1)
X2 = n2vAll.loc[df.db_n2v_idx].values
X3 = n2vAll.loc[df.ad_n2v_idx].values

# X = np.concatenate((X,X2,X3),axis=1)

In [ ]:
np.unique(foldNum,return_counts=True)

# Define Target Task

In [ ]:
y = df.delta3 >= 2

# Train and validate Logistic Regression Model
> Predict whether the mutation applied to db, represented by its mutpred2 features, will result in a score change at level 3 with the experiment's ad.

> The limitations here are that the MutPred2 features are independent of the AD protein even though the target value is a function of DB, MT, and AD

> The node2vec features are meaningless because the train and validation features are not in the same feature space

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score

In [ ]:
clf = RandomForestClassifier(n_estimators=500,n_jobs=20)
# clf = SVC(probability=True)
# clf = LogisticRegression()

clf.fit(X[foldNum==0],
        y[foldNum==0])

In [ ]:
plt.hist(clf.predict_proba(X[foldNum==0])[:,1])

In [ ]:
valpreds = clf.predict_proba(X[foldNum==1])[:,1]

In [ ]:
plt.hist(valpreds[y[foldNum==1]])
plt.hist(valpreds[~y[foldNum==1]],alpha=.5,color="red")

In [ ]:
roc_auc_score(y[foldNum==1],valpreds)